고정실,김가람,김득실,김영현,김충연,민병춘1,박주연1,박주연2,벌,안중훈,윤병시,이미우,임석봉,전창희,정광훈2,정금례,정용태,이귀임,정복연,김정한,정광훈1,조진욱

In [ ]:
# 대역별로 나눠서 TFR 계산하는 코드
print("90Hz까지의 대역별로 나누어 TFR을 시행하겠습니다.")

import pandas as pd
import numpy as np
import mne
import gc
import psutil
import threading
import time
from datetime import datetime

def check_memory():
    # Get memory usage
    memory_usage = psutil.Process().memory_info().rss / 1024 / 1024  # Convert to MB
    current_time = datetime.now().strftime('%H:%M:%S')
    print(f"[{current_time}] Memory usage: {memory_usage:.2f} MB")
    
    # Schedule next check in 5 minutes
    timer = threading.Timer(300, check_memory)  # 300 seconds = 5 minutes
    timer.daemon = True  # Allow program to exit even if timer is alive
    timer.start()

# Start memory monitoring
check_memory()

# 분석대상 이름 입력
name_list = list(input("분석대상 이름을 입력하세요. (여러명일 경우 쉼표로 구분)").split(','))

# bad channel 설정
bad_channel = {
    '박주연': 'C3, F4',
    '이미우': 'Fp1, Fp2, Fz, Cz, Pz',
    '임석봉': 'Fp1, Fp2',
    '김정한': 'Fp1, Fp2',
    '전창희': 'Fp1, F7',
    '정용태': 'Fp1, Fp2'
}

# 각 주파수 대역의 정보
freq_ranges = {
    'Gamma1': (30, 60)
}

# 분석 시작
for name in name_list:
    print(f"{name}의 TFR을 시행하겠습니다.")

    # bad channel 확인
    if name in bad_channel.keys():
        bad_channels_name = bad_channel[name]
        print(f"{name}의 bad channel은 {bad_channels_name}입니다.")
    else:
        bad_channels_name = ''
        print(f"{name}의 bad channel은 없습니다.")
    
    # 이름에 gamma 추가
    name_gamma = name + 'gamma'
    
    # TFR 시행
    print(f"{name}의 TFR을 시행하겠습니다.")
    
    # fif 파일 불러오기
    file_path = rf'D:\Mg_EEG\edf_files\{name_gamma}_7200_clean.fif'
    clean_data = mne.io.read_raw_fif(file_path, preload=True)
    print(f"{name}의 fif 파일을 성공적으로 불러왔습니다: {file_path}")

    # Delete data for a specific channel
    if bad_channels_name:
        bad_channels_list = [channel.strip() for channel in bad_channels_name.split(',')]
        clean_data.drop_channels(bad_channels_list)
    print(f"{name}의 bad channel을 제거하였습니다.")

    # Averaging by band and timing
    for band, (fmin, fmax) in freq_ranges.items():
        print(f"{name} {band}의 TFR을 계산 중 입니다.")
        tfr = clean_data.compute_tfr(method='multitaper', freqs=np.arange(fmin, fmax+1), tmin=0, tmax=7199, n_jobs=16, reject_by_annotation=False)
        print(f"{name} {band}의 TFR 계산이 완료되었습니다.")

        # Saving TFR file
        saving_path = rf'D:\Mg_EEG\tfr_gamma1\{name}{band}_7200_tfr.h5'
        tfr.save(saving_path, overwrite=False)
        print(f"TFR 파일이 성공적으로 저장되었습니다.")

        print(f"{name}{band}의 TFR 분석이 완료되었습니다.")

        # Delete tfr for memory
        del tfr
        print(f"{name}의 데이터를 메모리에서 삭제하였습니다.")
        gc.collect()
    
    print(f"{name}의 90Hz까지의 대역별로 나누어 TFR 분석이 완료되었습니다.")
    
    # Delete data for memory
    del clean_data
    gc.collect()

In [ ]:
import mne
tfr_gamma1 = mne.time_frequency.read_tfrs(rf"D:\Mg_EEG\tfr_gamma1\정광훈1Gamma1_7200_tfr.h5")

In [ ]:
tfr_gamma1.freqs